In [9]:
import pandas as pd
import duckdb
import re
import os
from typing import List, Dict, Any
from datetime import datetime
from difflib import SequenceMatcher
import warnings
warnings.filterwarnings('ignore')

print("✅ Libraries imported successfully!")

✅ Libraries imported successfully!


In [10]:
# Connect to existing DuckDB database
db_path = r"e:\ML SELF CODES\snaplife\patent_clinical_trial\patent_clinical\aact.duckdb"

if not os.path.exists(db_path):
    raise FileNotFoundError(f"DuckDB file not found: {db_path}. Please run aact_dataset_create.py first.")

conn = duckdb.connect(database=db_path, read_only=True)
print(f"✅ Connected to AACT DuckDB: {db_path}")

# Display database summary
tables_df = conn.execute("SHOW TABLES").fetchdf()
print(f"\n📊 Database contains {len(tables_df)} tables:")
for table_name in tables_df['name']:
    count = conn.execute(f"SELECT COUNT(*) as count FROM {table_name}").fetchone()[0]
    print(f"  {table_name:25}: {count:8,} rows")

✅ Connected to AACT DuckDB: e:\ML SELF CODES\snaplife\patent_clinical_trial\patent_clinical\aact.duckdb

📊 Database contains 49 tables:
  baseline_counts          :  218,212 rows
  baseline_measurements    : 2,627,555 rows
  brief_summaries          :  548,195 rows
  browse_conditions        : 3,712,505 rows
  browse_interventions     : 1,632,160 rows
  calculated_values        :  549,124 rows
  central_contacts         :  208,584 rows
  conditions               :  968,750 rows
  countries                :  752,441 rows
  design_group_interventions: 1,222,945 rows
  design_groups            : 1,003,790 rows
  design_outcomes          : 3,375,188 rows
  designs                  :  544,445 rows
  detailed_descriptions    :  548,195 rows
  documents                :   10,426 rows
  drop_withdrawals         :  548,013 rows
  eligibilities            :  548,195 rows
  facilities               : 3,291,989 rows
  facility_contacts        :  401,218 rows
  facility_investigators   :  217,201 r

In [11]:
class ClinicalTrialsSearchEngine:
    def __init__(self, connection):
        self.conn = connection
        
    def normalize_text(self, text: str) -> str:
        """Normalize text for better matching."""
        if not isinstance(text, str):
            return ""
        text = text.lower()
        text = re.sub(r'[^a-z0-9\s-]', ' ', text)
        text = re.sub(r'\s+', ' ', text)
        return text.strip()
    
    def search_comprehensive(self, query: str, 
                           phases: List[str] = None,
                           statuses: List[str] = None,
                           study_types: List[str] = None,
                           intervention_types: List[str] = None,
                           start_date_from: str = None,
                           start_date_to: str = None,
                           min_enrollment: int = None,
                           limit: int = 200) -> pd.DataFrame:
        """Comprehensive search with filters."""
        
        normalized_query = self.normalize_text(query)
        search_terms = normalized_query.split()
        
        # Create search conditions
        like_conditions = []
        for term in search_terms:
            condition = f"""
                (LOWER(s.brief_title) LIKE '%{term}%' OR 
                 LOWER(s.official_title) LIKE '%{term}%' OR
                 LOWER(i.name) LIKE '%{term}%' OR
                 LOWER(c.name) LIKE '%{term}%' OR
                 LOWER(bs.description) LIKE '%{term}%')
            """
            like_conditions.append(condition)
        
        where_clause = " AND ".join(like_conditions)
        
        # Add filters
        filters = []
        if phases:
            phase_list = "', '".join([p.upper() for p in phases])
            filters.append(f"UPPER(s.phase) IN ('{phase_list}')")
        
        if statuses:
            status_list = "', '".join([s.upper() for s in statuses])
            filters.append(f"UPPER(s.overall_status) IN ('{status_list}')")
        
        if study_types:
            type_list = "', '".join([st.upper() for st in study_types])
            filters.append(f"UPPER(s.study_type) IN ('{type_list}')")
        
        if intervention_types:
            int_type_list = "', '".join([it.upper() for it in intervention_types])
            filters.append(f"UPPER(i.intervention_type) IN ('{int_type_list}')")
        
        if start_date_from:
            filters.append(f"s.start_date >= '{start_date_from}'")
        
        if start_date_to:
            filters.append(f"s.start_date <= '{start_date_to}'")
        
        if min_enrollment:
            filters.append(f"CAST(s.enrollment AS INTEGER) >= {min_enrollment}")
        
        if filters:
            where_clause += " AND " + " AND ".join(filters)
        
        sql = f"""
        SELECT DISTINCT
            s.nct_id,
            s.brief_title,
            s.official_title,
            s.overall_status,
            s.phase,
            s.study_type,
            s.start_date,
            s.completion_date,
            s.enrollment,
            string_agg(DISTINCT i.name, ' | ') as interventions,
            string_agg(DISTINCT c.name, ' | ') as conditions,
            string_agg(DISTINCT sp.name, ' | ') as sponsors
        FROM studies s
        LEFT JOIN interventions i ON s.nct_id = i.nct_id
        LEFT JOIN conditions c ON s.nct_id = c.nct_id
        LEFT JOIN sponsors sp ON s.nct_id = sp.nct_id
        LEFT JOIN brief_summaries bs ON s.nct_id = bs.nct_id
        WHERE {where_clause}
        GROUP BY s.nct_id, s.brief_title, s.official_title, s.overall_status, 
                 s.phase, s.study_type, s.start_date, s.completion_date, s.enrollment
        ORDER BY s.start_date DESC
        LIMIT {limit}
        """
        
        try:
            result = self.conn.execute(sql).fetchdf()
            return result
        except Exception as e:
            print(f"❌ Search error: {e}")
            return pd.DataFrame()
    
    def get_trial_details(self, nct_id: str) -> Dict[str, Any]:
        """Get detailed information for a specific trial."""
        details = {}
        
        try:
            # Basic info
            basic_info = self.conn.execute(f"SELECT * FROM studies WHERE nct_id = '{nct_id}'").fetchdf()
            if not basic_info.empty:
                details['basic_info'] = basic_info.iloc[0].to_dict()
            
            # Interventions
            details['interventions'] = self.conn.execute(f"SELECT * FROM interventions WHERE nct_id = '{nct_id}'").fetchdf()
            
            # Conditions
            details['conditions'] = self.conn.execute(f"SELECT * FROM conditions WHERE nct_id = '{nct_id}'").fetchdf()
            
            # Sponsors
            details['sponsors'] = self.conn.execute(f"SELECT * FROM sponsors WHERE nct_id = '{nct_id}'").fetchdf()
            
            # Optional tables
            for table in ['brief_summaries', 'detailed_descriptions', 'facilities', 'outcomes']:
                try:
                    details[table] = self.conn.execute(f"SELECT * FROM {table} WHERE nct_id = '{nct_id}'").fetchdf()
                except:
                    details[table] = pd.DataFrame()
            
        except Exception as e:
            print(f"❌ Error getting trial details: {e}")
        
        return details
    
    def analyze_results(self, df: pd.DataFrame) -> Dict[str, Any]:
        """Analyze search results."""
        if df.empty:
            return {"message": "No results to analyze"}
        
        analysis = {
            'total_trials': len(df),
            'phase_distribution': df['phase'].value_counts().to_dict() if 'phase' in df.columns else {},
            'status_distribution': df['overall_status'].value_counts().to_dict() if 'overall_status' in df.columns else {},
            'study_type_distribution': df['study_type'].value_counts().to_dict() if 'study_type' in df.columns else {}
        }
        
        # Enrollment statistics
        if 'enrollment' in df.columns:
            enrollment_numeric = pd.to_numeric(df['enrollment'], errors='coerce')
            enrollment_stats = enrollment_numeric.describe()
            analysis['enrollment_stats'] = {
                'mean': enrollment_stats['mean'],
                'median': enrollment_stats['50%'],
                'min': enrollment_stats['min'],
                'max': enrollment_stats['max']
            }
        
        return analysis

# Initialize search engine
search_engine = ClinicalTrialsSearchEngine(conn)
print("✅ Clinical Trials Search Engine ready!")

✅ Clinical Trials Search Engine ready!


In [12]:
def search_clinical_trials(query: str,
                          phases: List[str] = None,
                          statuses: List[str] = None,
                          study_types: List[str] = None,
                          intervention_types: List[str] = None,
                          start_date_from: str = None,
                          start_date_to: str = None,
                          min_enrollment: int = None,
                          limit: int = 100,
                          save_results: bool = True) -> pd.DataFrame:
    """
    Main search function.
    
    Parameters:
    -----------
    query : str - Search term (drug name, condition, etc.)
    phases : List[str] - ['PHASE1', 'PHASE2', 'PHASE3']
    statuses : List[str] - ['COMPLETED', 'RECRUITING', 'ACTIVE_NOT_RECRUITING']
    study_types : List[str] - ['INTERVENTIONAL', 'OBSERVATIONAL']
    intervention_types : List[str] - ['DRUG', 'DEVICE', 'PROCEDURE']
    start_date_from : str - 'YYYY-MM-DD'
    start_date_to : str - 'YYYY-MM-DD'
    min_enrollment : int - Minimum participants
    limit : int - Max results
    save_results : bool - Save to CSV
    """
    
    print(f"🔍 Searching for: '{query}'")
    if phases: print(f"📋 Phases: {phases}")
    if statuses: print(f"📊 Statuses: {statuses}")
    if study_types: print(f"🔬 Study types: {study_types}")
    
    # Perform search
    results = search_engine.search_comprehensive(
        query=query,
        phases=phases,
        statuses=statuses,
        study_types=study_types,
        intervention_types=intervention_types,
        start_date_from=start_date_from,
        start_date_to=start_date_to,
        min_enrollment=min_enrollment,
        limit=limit
    )
    
    print(f"\n📊 Found {len(results)} trials")
    
    if not results.empty:
        # Show top results
        print(f"\n🎯 Top 10 results:")
        print("-" * 80)
        
        display_cols = ['nct_id', 'brief_title', 'phase', 'overall_status', 'start_date']
        for idx, row in results[display_cols].head(10).iterrows():
            print(f"{row['nct_id']}: {row['brief_title'][:60]}...")
            print(f"    Phase: {row['phase']}, Status: {row['overall_status']}, Start: {row['start_date']}")
            print()
        
        # Analyze results
        analysis = search_engine.analyze_results(results)
        print(f"\n📈 ANALYSIS:")
        print(f"Total: {analysis['total_trials']:,} trials")
        
        if 'phase_distribution' in analysis and analysis['phase_distribution']:
            print("\nPhase distribution:")
            for phase, count in analysis['phase_distribution'].items():
                print(f"  {phase}: {count} trials")
        
        if 'status_distribution' in analysis and analysis['status_distribution']:
            print("\nStatus distribution:")
            for status, count in list(analysis['status_distribution'].items())[:5]:
                print(f"  {status}: {count} trials")
        
        # Save results
        if save_results:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"clinical_trials_{query.replace(' ', '_')}_{timestamp}.csv"
            results.to_csv(filename, index=False)
            print(f"\n💾 Results saved to: {filename}")
    
    return results

def get_trial_summary(nct_id: str) -> None:
    """Get detailed summary for a specific trial."""
    print(f"📋 Trial Details: {nct_id}")
    print("=" * 60)
    
    details = search_engine.get_trial_details(nct_id)
    
    # Basic info
    if 'basic_info' in details and details['basic_info']:
        basic = details['basic_info']
        print(f"Title: {basic.get('official_title', 'N/A')}")
        print(f"Brief Title: {basic.get('brief_title', 'N/A')}")
        print(f"Phase: {basic.get('phase', 'N/A')}")
        print(f"Status: {basic.get('overall_status', 'N/A')}")
        print(f"Study Type: {basic.get('study_type', 'N/A')}")
        print(f"Start Date: {basic.get('start_date', 'N/A')}")
        print(f"Enrollment: {basic.get('enrollment', 'N/A')}")
        print()
    
    # Interventions
    if 'interventions' in details and not details['interventions'].empty:
        print("🔬 INTERVENTIONS:")
        for _, intervention in details['interventions'].iterrows():
            print(f"  - {intervention.get('intervention_type', 'N/A')}: {intervention.get('name', 'N/A')}")
        print()
    
    # Conditions
    if 'conditions' in details and not details['conditions'].empty:
        print("🏥 CONDITIONS:")
        for _, condition in details['conditions'].iterrows():
            print(f"  - {condition.get('name', 'N/A')}")
        print()
    
    # Sponsors
    if 'sponsors' in details and not details['sponsors'].empty:
        print("💰 SPONSORS:")
        for _, sponsor in details['sponsors'].iterrows():
            role = sponsor.get('lead_or_collaborator', 'N/A')
            print(f"  - {sponsor.get('name', 'N/A')} ({role})")
        print()

print("✅ Search functions ready!")

✅ Search functions ready!


In [13]:
# Example Search - Modify this cell for your searches
SEARCH_QUERY = "Alpha-2-macroglobulin"  # Change this to your search term

results = search_clinical_trials(
    query=SEARCH_QUERY
)

🔍 Searching for: 'Alpha-2-macroglobulin'

📊 Found 4 trials

🎯 Top 10 results:
--------------------------------------------------------------------------------
NCT03656575: Reduction of Pro-Inflammatory Synovial Fluid Biomarkers in O...
    Phase: PHASE1, Status: COMPLETED, Start: 2017-11-01

NCT02210468: APIC-CF Therapy for Mild to Moderate Osteoarthritis of the K...
    Phase: PHASE1/PHASE2, Status: UNKNOWN, Start: 2015-05-31

NCT03307876: Injection of an Autologous A2M Concentrate Alleviates Back P...
    Phase: None, Status: COMPLETED, Start: 2014-04-30

NCT01613833: Serum and Synovium Protease Inhibitor Levels in Primary and ...
    Phase: None, Status: SUSPENDED, Start: 2012-03-31


📈 ANALYSIS:
Total: 4 trials

Phase distribution:
  PHASE1: 1 trials
  PHASE1/PHASE2: 1 trials

Status distribution:
  COMPLETED: 2 trials
  UNKNOWN: 1 trials
  SUSPENDED: 1 trials

💾 Results saved to: clinical_trials_Alpha-2-macroglobulin_20250913_192651.csv

📊 Found 4 trials

🎯 Top 10 results:
-------

In [14]:
# Get detailed information about the first trial from results
if not results.empty:
    sample_nct_id = results.iloc[0]['nct_id']
    get_trial_summary(sample_nct_id)
else:
    print("No trials found to display details for.")

📋 Trial Details: NCT03656575
Title: Reduction of Pro-Inflammatory Synovial Fluid Biomarkers in Osteoarthritis of the Knee With Alpha-2 Macroglobulin: A Randomized Controlled Trial
Brief Title: Reduction of Pro-Inflammatory Synovial Fluid Biomarkers in Osteoarthritis of the Knee With Alpha-2 Macroglobulin
Phase: PHASE1
Status: COMPLETED
Study Type: INTERVENTIONAL
Start Date: 2017-11-01
Enrollment: 75

🔬 INTERVENTIONS:
  - BIOLOGICAL: synovial fluid biomarker concentrations

🏥 CONDITIONS:
  - Osteoarthritis

💰 SPONSORS:
  - NYU Langone Health (lead)



In [15]:
def save_comprehensive_trial_data(results_df: pd.DataFrame, filename: str = None) -> str:
    """
    Save comprehensive trial data with all detailed information to CSV.
    
    Parameters:
    -----------
    results_df : pd.DataFrame - Results from search_clinical_trials()
    filename : str - Optional custom filename
    
    Returns:
    --------
    str : Path to saved CSV file
    """
    
    if results_df.empty:
        print("❌ No results to save")
        return None
    
    print(f"📊 Processing {len(results_df)} trials for comprehensive data export...")
    
    comprehensive_data = []
    
    for idx, row in results_df.iterrows():
        nct_id = row['nct_id']
        print(f"Processing {nct_id} ({idx+1}/{len(results_df)})")
        
        # Get detailed trial information
        details = search_engine.get_trial_details(nct_id)
        
        # Extract basic information
        basic_info = details.get('basic_info', {})
        
        # Build comprehensive record
        record = {
            'NCT Number': nct_id,
            'Study Title': basic_info.get('official_title', basic_info.get('brief_title', 'N/A')),
            'Study URL': f"https://clinicaltrials.gov/study/{nct_id}",
            'Study Status': basic_info.get('overall_status', 'N/A'),
            'Brief Summary': '',
            'Conditions': '',
            'Interventions': '',
            'Sponsor': '',
            'Collaborators': '',
            'Phases': basic_info.get('phase', 'N/A'),
            'Study Type': basic_info.get('study_type', 'N/A'),
            'Start Date': basic_info.get('start_date', 'N/A'),
            'Completion Date': basic_info.get('completion_date', 'N/A'),
            'Enrollment': basic_info.get('enrollment', 'N/A'),
            'Primary Completion Date': basic_info.get('primary_completion_date', 'N/A'),
            'Last Update Posted': basic_info.get('last_update_posted', 'N/A')
        }
        
        # Get brief summary
        if 'brief_summaries' in details and not details['brief_summaries'].empty:
            summaries = details['brief_summaries']['description'].tolist()
            record['Brief Summary'] = ' '.join([str(s) for s in summaries if pd.notna(s)])
        
        # Get conditions
        if 'conditions' in details and not details['conditions'].empty:
            conditions = details['conditions']['name'].tolist()
            record['Conditions'] = '|'.join([str(c) for c in conditions if pd.notna(c)])
        
        # Get interventions with types
        if 'interventions' in details and not details['interventions'].empty:
            interventions = []
            for _, intervention in details['interventions'].iterrows():
                int_type = intervention.get('intervention_type', 'N/A')
                int_name = intervention.get('name', 'N/A')
                interventions.append(f"{int_type}: {int_name}")
            record['Interventions'] = '|'.join(interventions)
        
        # Get sponsors and collaborators
        if 'sponsors' in details and not details['sponsors'].empty:
            lead_sponsors = []
            collaborators = []
            
            for _, sponsor in details['sponsors'].iterrows():
                sponsor_name = sponsor.get('name', 'N/A')
                role = sponsor.get('lead_or_collaborator', '').upper()
                
                if role == 'LEAD':
                    lead_sponsors.append(sponsor_name)
                elif role == 'COLLABORATOR':
                    collaborators.append(sponsor_name)
                else:
                    # If role is unclear, add to sponsors
                    lead_sponsors.append(sponsor_name)
            
            record['Sponsor'] = '|'.join(lead_sponsors)
            record['Collaborators'] = '|'.join(collaborators)
        
        comprehensive_data.append(record)
    
    # Create DataFrame
    comprehensive_df = pd.DataFrame(comprehensive_data)
    
    # Generate filename if not provided
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"comprehensive_clinical_trials_{timestamp}.csv"
    
    # Save to CSV
    comprehensive_df.to_csv(filename, index=False, encoding='utf-8')
    
    print(f"\n✅ Comprehensive trial data saved to: {filename}")
    print(f"📊 Exported {len(comprehensive_df)} trials with complete information")
    print(f"📋 Columns: {', '.join(comprehensive_df.columns)}")
    
    # Show sample of saved data
    print(f"\n🔍 Sample of saved data:")
    print("-" * 80)
    for idx, row in comprehensive_df.head(3).iterrows():
        print(f"NCT: {row['NCT Number']}")
        print(f"Title: {row['Study Title'][:100]}...")
        print(f"Status: {row['Study Status']}, Phase: {row['Phases']}")
        print(f"Conditions: {row['Conditions'][:80]}...")
        print()
    
    return filename

# Save comprehensive data for current results
if 'results' in locals() and not results.empty:
    comprehensive_file = save_comprehensive_trial_data(results)
else:
    print("⚠️ No search results available. Run a search first to save comprehensive data.")
    print("Example: results = search_clinical_trials('your_search_term')")

📊 Processing 4 trials for comprehensive data export...
Processing NCT03656575 (1/4)
Processing NCT02210468 (2/4)
Processing NCT03307876 (3/4)
Processing NCT01613833 (4/4)
Processing NCT03307876 (3/4)
Processing NCT01613833 (4/4)

✅ Comprehensive trial data saved to: comprehensive_clinical_trials_20250913_192653.csv
📊 Exported 4 trials with complete information
📋 Columns: NCT Number, Study Title, Study URL, Study Status, Brief Summary, Conditions, Interventions, Sponsor, Collaborators, Phases, Study Type, Start Date, Completion Date, Enrollment, Primary Completion Date, Last Update Posted

🔍 Sample of saved data:
--------------------------------------------------------------------------------
NCT: NCT03656575
Title: Reduction of Pro-Inflammatory Synovial Fluid Biomarkers in Osteoarthritis of the Knee With Alpha-2 M...
Status: COMPLETED, Phase: PHASE1
Conditions: Osteoarthritis...

NCT: NCT02210468
Title: A Phase I/II, Multicenter, Double-Blind, Placebo-controlled Safety and Efficacy Stu